In [34]:
import os
import cv2
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.datasets import DatasetFolder
from torchvision import transforms, utils
import numpy as np
import pickle
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
from torchsummary import summary

# Pollen-Detection-Network

### Network

In [35]:
class FCNN(nn.Module):
    """
    This is a class for fully convolutional neural networks.
    
    It is a subclass of the Module class from torch.nn.
    See the torch.nn documentation for more information.
    """
    
    def __init__(self):
        """
        The constructor for FCNN class. The internal states of the network are initialized. 
        """
        
        super(FCNN, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv6 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=1, stride=1)
        
        
    def forward(self, data):
        """
        Defines the computation performed at every call.
        
        Parameters:
            data (torch.Tensor): The input that is evaluated by the network. 
                The network expects the input to be of 4 dimensions.
            
        Returns:
            x (torch.Tensor): The output of the network after evaluating it on the given input.
        """
        
        x = F.relu(self.conv0(data))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.sigmoid(self.conv6(x))
        return x

### Validation

In [50]:
def validate(network, testloader):
    """
    Calculates the mean validation loss, the F1-Score and the accuracy of the given network on the testloader data.
    
    Parameters:
        network (nn.Module): The network to validate. 
        testloader (torch.utils.data.DataLoader): Contains the data which is used to validate the network. 
        
    Returns:
        validation_loss (float): The mean loss per image with regards to a fixed criterion.
        F1_Score (float): The harmonic mean of the precision and the recall of the given network
            on the given validation data.
        correct / total (float): The percentage of correctly classified samples of the 
            total number of validation samples.
        
    """
    
    total = 0
    correct = 0
    true_positive = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0

    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        validation_loss = 0
        for i, sample in enumerate(testloader):
#         for i, (inputs, labels) in enumerate(testloader):
            
            inputs = sample[0].to(device)
            labels = sample[1].to(device)
#             inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            loss = criterion(outputs,labels)
            validation_loss += loss.item()
            predicted = (outputs >= threshold) # Predicted is a tensor of booleans 
            total += labels.size(0)
            predicted = predicted.view(predicted.size(0)) 
            b_labels = labels != 0
            correct += (predicted == b_labels).sum().item()
            true_positive += (predicted & b_labels).sum().item()
            false_negative += (np.logical_not(np.array(torch.Tensor.cpu(predicted))) & np.array(torch.Tensor.cpu(b_labels))).sum()
            
        f_validation_loss = validation_loss / total 
        true_negative = correct - true_positive
        false_positive = total - correct - false_negative
        
        try: 
            recall = true_positive / (true_positive + false_negative)
        except ZeroDivisionError:
            recall = 0
        try:
            precision = true_positive / (true_positive + false_positive)
        except ZeroDivisionError:
            precision = 0
        try:
            F1_score = 2 * (precision * recall) / (precision + recall)
        except ZeroDivisionError:
            F1_score = 0
            
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    print('F1 Score of the network on the test images: %f' % F1_score)
    return f_validation_loss, F1_score, (correct / total)

### Training

In [53]:
def train(network, trainloader, validloader, ep, criterion, optimizer, print_interval):
    """
    Train a neural network.
    
    
    Parameters:
        network (nn.Module): The network which should be trained.
        trainloader (torch.utils.data.DataLoader): Contains the data to be used to train the network.
        ep (int): The number of epochs the network will be trained for.
        criterion (): The loss function used for training (should be the same for validation).
        optimizer (): The specification how the weights are optimized.
        print_interval (int): The number of batches after which the function prints the loss for one batch.
    
    Returns:
        train_losses ([float]): The mean of the loss for one train sample after every epoch. 
        validation_losses ([float]): The mean of the loss for one validation sample after every epoch.
        F1 ([float]): The F1-Score of the validation data after every epoch.
        accuracy ([float]): The accuracy of the network on the validation data after every epoch.
    """
    
    losses = [] # Mean of loss/image in every epoch of training 
    validation_losses = []
    F1 = []
    accuracy = []
    print('Performance of the untrained network:')
    validate(network, validloader)
    
    for epoch in range(ep):

        loss_stats = 0.0
        total = 0
        
        running_loss = 0.0
    
        for i, sample in enumerate(trainloader):
#         for i, (inputs, labels) in enumerate(trainloader):
            
            inputs = sample[0].to(device)
            labels = sample[1].to(device)
#             inputs, labels = inputs.to(device), labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            #print(labels.shape)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            #print(outputs.shape)
            loss = criterion(outputs, labels)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
            total += labels.shape[0]
            
            #Some statistics:
            running_loss += loss.item()
            
            loss_stats += loss.item()
            
            if i % print_interval == print_interval - 1:    # print every x mini-batches (loss for one batch)
                print('[Epoch: %d, Batch: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
        
        losses.append(loss_stats / total) 
        #Test the network using the validation set after every epoch of training.
        val_loss_curr, F1_curr, accuracy_curr = validate(network, validloader)
        validation_losses.append(val_loss_curr)
        F1.append(F1_curr)
        accuracy.append(accuracy_curr)
        
    print('Finished Training')
    return losses, validation_losses, F1, accuracy

### Create Dataset

In [38]:
class LabelStrengthDataset(DatasetFolder):
    """A generic data loader where the samples are arranged in this way: ::
        root/class_x/smpl1
        root/class_x/smpl2
        
        root/class_y/smpl3
        root/class_y/smpl4
        p
        with a single dictionary containing all samples as keys and their labelstrength as values
        
    Args:
        root (String):    path to root
        ls_file (String): path to pickle file containing the dictionary
    
    Attributes:
        root: path to root
        ls: dictionary with image names as keys and labelstrengths as values
        transform: transformations applied on samples
        samples: List of tuples containing a path to an image and its label strength
    """
    
    def __init__(self, root, ls_file, transform=None):
        with open(ls_file, 'rb') as pklfile:
            self.ls = pickle.load(pklfile)
        self.root = root
        self.transform = transform
        self.samples = self.make_dataset()
    
    def __len__(self):
        return len(self.ls)

    def __getitem__(self,index):
        img_path = self.samples[index][0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        np_image = np.array(image)
        t_image = torch.Tensor(np_image)
        labelstrength = self.samples[index][1]
        sample = (t_image, labelstrength)
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def make_dataset(self):
        """Creates List of Tulpes containing path to img and its labelstrength"""
        print("Create Dataset..")
        instances = []
        for root, folders, files in os.walk(self.root):
            for file in files:
                if file.lower().endswith('.png'):
                    img_path = os.path.join(root,file)
                    label = self.ls[file]
#                     t_label = torch.tensor(label)
                    item = img_path, label
                    instances.append(item)
        print("Dataset created.")
        return instances

In [39]:
cwd = os.getcwd()
cwd

'/home/tkf/Desktop/dev/pollen-detection'

In [40]:
root = os.path.join(cwd, 'Datasets/PollenData/')
pickle_dir = os.path.join(root, 'labelstrength.pickle')

In [28]:
dataset = LabelStrengthDataset(root, pickle_dir)

Create Dataset..
Dataset created.


### Create Train- and Testsets

In [41]:
batch_size = 32
train_ratio = .8
shuffle_dataset = True
seed = np.random.randint
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(train_ratio * dataset_size))

if shuffle_dataset:
    np.random.shuffle(indices)

train_indices, valid_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

# Train Network

#### Now we can train our network with given data

#### Initialise the network

In [42]:
# Setup wether gpu or cpu should be used for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a new instance of the FCNN class. This will be our network.      
pollen_network = FCNN().to(device)
    
# Choose optimizer, criterion, number of epochs and threshold for the network to accept or dismiss a computated input
criterion = nn.MSELoss()
optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)
ep = 10
threshold = 0.8

### Test untrained network

In [49]:
val_loss_curr, F1_curr, accuracy_curr = validate(pollen_network, valid_loader)

Inputs  <class 'torch.Tensor'> tensor([[[ 25.,  26.,  30.,  ...,  44.,  48.,  49.],
         [ 26.,  27.,  30.,  ...,  46.,  48.,  49.],
         [ 24.,  26.,  29.,  ...,  44.,  49.,  54.],
         ...,
         [108., 109., 108.,  ...,  28.,  28.,  28.],
         [110., 112., 108.,  ...,  26.,  28.,  30.],
         [110., 114., 108.,  ...,  30.,  28.,  30.]],

        [[255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         ...,
         [255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.]],

        [[242., 255., 255.,  ..., 246., 242., 252.],
         [247., 242., 245.,  ..., 255., 253., 251.],
         [247., 250., 239.,  ..., 250., 255., 255.],
         ...,
         [255., 255., 255.,  ..., 251., 238., 250.],
         [255., 246., 255.,  ..., 239., 251., 255.],
         [249., 255., 

### Train Network

In [54]:
train_losses, val_losses, F1_scores, accuracies = train(pollen_network, train_loader, valid_loader, ep, criterion, optimizer, 32)

Performance of the untrained network:
Accuracy of the network on the test images: 85 %
F1 Score of the network on the test images: 0.003825


RuntimeError: expected dtype Double but got dtype Float (validate_dtype at /pytorch/aten/src/ATen/native/TensorIterator.cpp:143)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x46 (0x7f548dbbe536 in /home/tkf/.local/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: at::TensorIterator::compute_types() + 0xce3 (0x7f547ff04483 in /home/tkf/.local/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #2: at::TensorIterator::build() + 0x44 (0x7f547ff06e64 in /home/tkf/.local/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #3: at::native::mse_loss_backward_out(at::Tensor&, at::Tensor const&, at::Tensor const&, at::Tensor const&, long) + 0x193 (0x7f547fd54e93 in /home/tkf/.local/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #4: <unknown function> + 0x10b70b7 (0x7f54801810b7 in /home/tkf/.local/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #5: at::native::mse_loss_backward(at::Tensor const&, at::Tensor const&, 

#### Save trained Network

In [12]:
# save the networks parameters to have the possibility to reload them later:
cur_dir = os.getcwd()
save_path = cur_dir + '/pollennet.pt'
torch.save(pollen_network.state_dict(), save_path)

# save the corresponding losses, F1-Scores and Accuracies using pickle:

with open('train_losses_gpu.obj', 'wb') as train_losses_file:
    pickle.dump(train_losses, train_losses_file)

with open('val_losses_gpu.obj', 'wb') as val_losses_file:
    pickle.dump(val_losses, val_losses_file)
    
with open('F1_gpu.obj', 'wb') as F1_file:
    pickle.dump(F1_scores, F1_file)
    
with open('accuracies_gpu.obj', 'wb') as accuracies_file:
    pickle.dump(accuracies, accuracies_file)

In [ ]:
train_losses_2, val_losses_2, F1_scores_2, accuracies_2 = train(pollen_network, trainloader, ep, criterion, optimizer, 32)

Performance of the untrained network:
Accuracy of the network on the test images: 97 %
F1 Score of the network on the test images: 0.879973
[Epoch: 1, Batch:    32] loss: 0.018
[Epoch: 1, Batch:    64] loss: 0.017
[Epoch: 1, Batch:    96] loss: 0.013
[Epoch: 1, Batch:   128] loss: 0.017
[Epoch: 1, Batch:   160] loss: 0.017
[Epoch: 1, Batch:   192] loss: 0.016
[Epoch: 1, Batch:   224] loss: 0.011
[Epoch: 1, Batch:   256] loss: 0.015
[Epoch: 1, Batch:   288] loss: 0.011
[Epoch: 1, Batch:   320] loss: 0.011
[Epoch: 1, Batch:   352] loss: 0.013
[Epoch: 1, Batch:   384] loss: 0.018
[Epoch: 1, Batch:   416] loss: 0.019
[Epoch: 1, Batch:   448] loss: 0.017
[Epoch: 1, Batch:   480] loss: 0.019
[Epoch: 1, Batch:   512] loss: 0.014
[Epoch: 1, Batch:   544] loss: 0.017
[Epoch: 1, Batch:   576] loss: 0.015
[Epoch: 1, Batch:   608] loss: 0.016
[Epoch: 1, Batch:   640] loss: 0.020
[Epoch: 1, Batch:   672] loss: 0.012
[Epoch: 1, Batch:   704] loss: 0.014
[Epoch: 1, Batch:   736] loss: 0.020
[Epoch: 1

[Epoch: 2, Batch:   864] loss: 0.018
[Epoch: 2, Batch:   896] loss: 0.019
[Epoch: 2, Batch:   928] loss: 0.012
[Epoch: 2, Batch:   960] loss: 0.012
[Epoch: 2, Batch:   992] loss: 0.019
[Epoch: 2, Batch:  1024] loss: 0.015
[Epoch: 2, Batch:  1056] loss: 0.015
[Epoch: 2, Batch:  1088] loss: 0.013
[Epoch: 2, Batch:  1120] loss: 0.011
[Epoch: 2, Batch:  1152] loss: 0.015
[Epoch: 2, Batch:  1184] loss: 0.011
[Epoch: 2, Batch:  1216] loss: 0.013
[Epoch: 2, Batch:  1248] loss: 0.012
[Epoch: 2, Batch:  1280] loss: 0.025
[Epoch: 2, Batch:  1312] loss: 0.015
[Epoch: 2, Batch:  1344] loss: 0.015
[Epoch: 2, Batch:  1376] loss: 0.012
[Epoch: 2, Batch:  1408] loss: 0.013
[Epoch: 2, Batch:  1440] loss: 0.011
[Epoch: 2, Batch:  1472] loss: 0.020
[Epoch: 2, Batch:  1504] loss: 0.010
[Epoch: 2, Batch:  1536] loss: 0.018
[Epoch: 2, Batch:  1568] loss: 0.016


## Old Stuff

### Create Data

In [ ]:
# def load_pollen_data(dir):
#     np_pollen_data = []     # single date will be [img,label]
#     for folder in next(os.walk(dir))[1]:
#         if folder == 'p':
#             label = 1
#         else:
#             label = 0
#         parent_path = os.path.join(dir, folder)
#         for file in os.listdir(parent_path):
#             if '.png' in file:
#                 try:
#                     path = os.path.join(parent_path, file)
#                     img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#                     np_pollen_data.append([np.array(img), label])
#                 except Exception as e:
#                     print(folder, file, str(e))

#     data = np.random.shuffle(np_pollen_data)
#     print("Data created.")
#     return np_pollen_data

### Create Train and Test Data

In [ ]:
# def create_test_val_sets(data, ratio=0.8):
#     """
#     Splits the data into a training and a validation data set.
    
#     Parameters:
#         data ([[obj, obj]]): A list of lists, which contain a sample and the correponding label.
#         ratio (float): The ratio between the size of the validation and the training data set.
        
#     Returns:
#         train_x ([obj]): The samples of the training data set.
#         train_y ([obj]): The corresponding labels of the training data set.
#         valid_x ([obj]): The samples of the validation data set.
#         valid_y ([obj]): The corresponding labels of the validation data set.
    
#     """
#     images = []
#     labels = []

#     for i in range(len(data)):
#         images.append(data[i][0])
#         labels.append(data[i][1])
    
#     train_x = images[:int(len(images)*ratio)]
#     train_y = labels[:int(len(images)*ratio)]
#     valid_x = images[int(len(images)*ratio):]
#     valid_y = labels[int(len(images)*ratio):]
    
#     print("Sets created")
#     return train_x, train_y, valid_x, valid_y

#### Load Data

In [ ]:
# # Load Data
# cur_dir = os.getcwd()
# pollen_path = os.path.join(cur_dir, 'Datasets/PollenData64')
# pollendata = load_pollen_data(pollen_path)

In [ ]:
# # Split into sets
# train_x, train_y, valid_x, valid_y = create_test_val_sets(pollendata, 0.9)

# pollendata = []

# # Our neural network expects a dataloader
# trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(train_x),torch.Tensor(train_y)),batch_size=32)
# validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(valid_x),torch.Tensor(valid_y)),batch_size=1)

# # Save dataloader for later use
# #torch.save(trainloader, 'trainloader.pth')
# #torch.save(validloader, 'validloader.pth')

#### Initialise the network

In [ ]:
# # Setup wether gpu or cpu should be used for computations
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Create a new instance of the FCNN class. This will be our network.      
# pollen_network = FCNN().to(device)

# # Choose optimizer, criterion, number of epochs and threshold for the network to accept or dismiss a computated input
# criterion = nn.MSELoss()
# optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)
# ep = 10
# threshold = 0.8